# Spotify Recommendation System Project

**Purpose:** Build a scalable music recommendation system using PySpark and Spotify datasets.

**Group members:**

* Raakin Bhatti
* Aneesh Bulusu
* Walid Farhat
* Long Nguyen
* Strahinja Radakovic

# Introduction

**What is our big data problem? What is our goal?**
* Build a scalable music recommendation system using a large dataset of songs and their audio features from Spotify.
* Given an input (e.g., a song's name), the algorithm will predict and recommend related songs based on their relevance and similarity to the input.
* Use Spark/PySpark for processing large-scale data and developing machine learning algorithms.

**Why we chose building Recommendation Systems:**
* Recommendation systems are widely used in modern digital platforms to enhance user experience (e.g., Spotify, Netflix, Amazon).
* Help users discover relevant content, which helps to increase engagement and satisfaction.

**Why Spotify dataset?**
* Spotify is a leading music streaming platform which has rich data on songs, audio features, and artists.
* The dataset provides an opportunity to analyze music preferences and recommend personalized songs or playlists.

**What is a big data challenge?**
* The solution involves leveraging PySpark’s distributed computing capabilities to handle, transform, and analyze large volumes of data efficiently.
* PySpark is well-suited for handling and analyzing big data with its distributed computing capabilities.

**Broader applications:**
* Insights from the project can extend to other recommendation systems.
* Demonstrates the integration of big data tools and machine learning for real-world applications.


# Setting up Spark

This section is optional. In case that you have not installed Spark, Hadoop, etc. in your local machine, then this part will help setting up Spark in the Jupyter Notebook for running.

In [1]:
## OPTIONAL: Setting up Spark in Jupyter Notebook

# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget https://dlcdn.apache.org/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
# !tar -xvf spark-3.5.3-bin-hadoop3.tgz
# !pip install findspark
# 
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.5.3-bin-hadoop3"
# 
# import findspark
# findspark.init()
# findspark.find()

In [2]:
## OPTIONAL: Test if PySpark is ready to go

# from pyspark.sql import SparkSession
# 
# spark = SparkSession.builder.appName("Test").getOrCreate()
# print(f"Spark version:", spark.version)
# 
# spark.stop()

## expected result: 3.5.3 or similar

# Data Collection

**Required Tasks:**
* Load the dataset into a PySpark DataFrame.
* Verify the dataset schema and check if the data is loaded correctly.

**Output:** A PySpark DataFrame loaded and ready for processing, with the schema verified.


### How we get the dataset?

We extracted the dataset from Spotify using Spotify API and the `Spotipy` library in Python.

Attached the Python files for extraction and transformation (from JSON to CSV). Please note that these files are for references only, because in order to run those files, you will have to set up a virtual environment.

### Loading datasets

In [4]:
import findspark
findspark.init()
findspark.find()

'C:\\Program Files\\spark-3.5.3'

In [5]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count
import pandas as pd
import matplotlib.pyplot as plt

# initialize a SparkSession
spark = SparkSession.builder \
    .appName("SpotifyRecommendationSystem") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

# load dataset into a DataFrame
file_path = "./dataset/spotify_dataset.csv" # Note: The dataset file is too large (>250 MB) to commit to a GitHub repo.
spotify_df = spark.read.csv(file_path, header=True, inferSchema=True)

# display the first few rows
spotify_df.show(5)


+---+-------------+----------------+--------------------+----------+----+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+-----------+--------------+
|_c0|  artist_name|      track_name|            track_id|popularity|year|   genre|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|duration_ms|time_signature|
+---+-------------+----------------+--------------------+----------+----+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+-----------+--------------+
|  0|   Jason Mraz| I Won't Give Up|53QF56cjZA9RTuuMZ...|        68|2012|acoustic|       0.483| 0.303|  4| -10.058|   1|     0.0429|       0.694|             0.0|   0.115|  0.139|133.406|   240166.0|           3.0|
|  1|   Jason Mraz|93 Million Miles|1s8tP3jP4GZcyHDsj...|        50|2012|acoustic|       0.572| 0.454|  3| -10.286|   1|     0.0258|       0

# Data Inspection and Validation

In this section, we will do the following tasks:

* Check the data schema with column names and data types.
* Convert data types if needed.
* Check for the summary statistics of the dataset.
* Check for missing values. Handle missing values properly.
* Check for outliers. Handle the outliers.
* Check for distinct values.

In [6]:
# print schema to verify column names and data types
spotify_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- track_id: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- year: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- duration_ms: double (nullable = true)
 |-- time_signature: double (nullable = true)



**Data type conversins:**

From [Spotify's API documentation](https://developer.spotify.com/documentation/web-api/reference/get-audio-features) about track's audio features, each feature has its own meaning and data types. So we will convert the data types of features in our dataset to match Spotify's documentation.

In [7]:
# casting columns to their appropriate data types as per Spotify's documentation
spotify_df = spotify_df \
    .withColumn("popularity", col("popularity").cast("int")) \
    .withColumn("year", col("year").cast("int")) \
    .withColumn("danceability", col("danceability").cast("float")) \
    .withColumn("energy", col("energy").cast("float")) \
    .withColumn("key", col("key").cast("int")) \
    .withColumn("loudness", col("loudness").cast("float")) \
    .withColumn("mode", col("mode").cast("int")) \
    .withColumn("speechiness", col("speechiness").cast("float")) \
    .withColumn("acousticness", col("acousticness").cast("float")) \
    .withColumn("instrumentalness", col("instrumentalness").cast("float")) \
    .withColumn("liveness", col("liveness").cast("float")) \
    .withColumn("tempo", col("tempo").cast("float")) \
    .withColumn("time_signature", col("time_signature").cast("int"))

# re-check the updated schema
spotify_df.printSchema()


root
 |-- _c0: integer (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- track_id: string (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- genre: string (nullable = true)
 |-- danceability: float (nullable = true)
 |-- energy: float (nullable = true)
 |-- key: integer (nullable = true)
 |-- loudness: float (nullable = true)
 |-- mode: integer (nullable = true)
 |-- speechiness: float (nullable = true)
 |-- acousticness: float (nullable = true)
 |-- instrumentalness: float (nullable = true)
 |-- liveness: float (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: float (nullable = true)
 |-- duration_ms: double (nullable = true)
 |-- time_signature: integer (nullable = true)



In [8]:
# count total rows in the dataset
total_rows = spotify_df.count()
print(f"Total rows in the dataset: {total_rows}")

Total rows in the dataset: 1159764


Remarks: The dataset contains 1,159,764 rows, which is quite large, indicating the need for big data tools like PySpark.

**Check for Summary Statistics:**

Next, we want to check for the summary statistics of the dataset. In [Spark](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.describe.html), if using `.describe()` method, it will by default calculate the stats for all columns including both numerical and non-numerical (string) columns. Therefore, we have to filter out the columns based on their data types as follows.

In [12]:
# filter numeric columns using dtypes
numeric_columns = [name for name, dtype in spotify_df.dtypes if dtype in ('int', 'bigint', 'double', 'float', 'decimal')]

# select only numeric columns
numeric_df = spotify_df.select(*numeric_columns)

# show summary statistics for numerical columns only
numeric_df.describe().show()

+-------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+
|summary|              _c0|        popularity|              year|     danceability|            energy|               key|          loudness|             mode|        speechiness|       acousticness|   instrumentalness|           liveness|            valence|             tempo|       duration_ms|    time_signature|
+-------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+
|  count|          1159764|           1158091|      

**Remarks for summary statistics:**

* `popularity`: Ranges from 0 to 100, indicating a reasonable scale for popularity.
* `year`: The dataset includes tracks from 0 to 2023. The value 0 seems anomalous and might need further investigation.
* Other features like `danceability`, `energy`, `tempo`, and `duration_ms` have a wide range of values, which may need normalization or standardization for machine learning.
* Potential outliers: Columns like `tempo` (min = -24.073) and `loudness` (min = -58.1) have unusual values that might indicate outliers or data entry issues.

In [13]:
# check for missing values in each column
missing_values = spotify_df.select([
    count(when(col(c).isNull(), c)).alias(c) for c in spotify_df.columns
])
missing_values.show()

+---+-----------+----------+--------+----------+----+-----+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+-----------+--------------+
|_c0|artist_name|track_name|track_id|popularity|year|genre|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|tempo|duration_ms|time_signature|
+---+-----------+----------+--------+----------+----+-----+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+-----------+--------------+
|  0|          0|         0|       0|      1673| 623|    0|        1184|   408|162|      94|  33|         16|           8|               3|       2|      0|    0|          0|             0|
+---+-----------+----------+--------+----------+----+-----+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+-----------+--------------+



**Remarks for missing values:**

* Columns with missing data: popularity (1,673 missing values), year (623), and others such as  danceability, energy, key, loudness, and mode.
* Given the dataset has over 1.15M rows, the proportion of missing data is extremely small (less than 0.1% for all affected columns).
* In this case, we will drop rows with missing values in features that are critical for our recommendation system.

In [15]:
# drop rows with missing data in critical columns
columns_to_check = ['popularity', 'year', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness']

spotify_df_clean = spotify_df.na.drop(subset=columns_to_check)

# verify the number of rows after dropping
cleaned_rows = spotify_df_clean.count()
print(f"Rows after dropping missing data: {cleaned_rows}")

dropped_rows = total_rows - cleaned_rows
print(f"Number of rows dropped: {dropped_rows}")


Rows after dropping missing data: 1158091
Number of rows dropped: 1673


**Anomalies and Outliers**:

We will investigate the folloiwng things:

* Check for distinct values of `year` column to make sure no abnormal values (such as 1500).
* 



In [19]:
# get distinct years and their frequencies
year_distribution = spotify_df_clean.groupBy("year").count().orderBy("year")

year_distribution.show(50)

+----+-----+
|year|count|
+----+-----+
|2000|43944|
|2001|42316|
|2002|42084|
|2003|42250|
|2004|43293|
|2005|43708|
|2006|45419|
|2007|45920|
|2008|47336|
|2009|46810|
|2010|46818|
|2011|46381|
|2012|54725|
|2013|53105|
|2014|53120|
|2015|51569|
|2016|40246|
|2017|56171|
|2018|56541|
|2019|55739|
|2020|55035|
|2021|53529|
|2022|53637|
|2023|38395|
+----+-----+



Remarks for `year`: The years range from 2000 to 2023, which are expected and normal.

**Check for anomalies in numerical columns:**

Compare data against the value ranges specified in the [Spotify documentation](https://developer.spotify.com/documentation/web-api/reference/get-audio-features):
* acousticness: [0, 1]
* danceability: [0, 1]
* energy: [0, 1] 
* instrumentalness: [0, 1]
* key: [-1, 11]
* liveness: [0, 1]
* mode: {0, 1}
* speechiness: [0, 1]
* tempo: Typically [1, 250], (note: this is a typical range; Spotify docs don't strictly enforce this.)
* time_signature: [3, 7]
* valence: [0, 1]

In [20]:
# define the range checks for each feature
range_checks = {
    "acousticness": (0, 1),
    "danceability": (0, 1),
    "energy": (0, 1),
    "instrumentalness": (0, 1),
    "key": (-1, 11),
    "liveness": (0, 1),
    "mode": (0, 1),
    "speechiness": (0, 1),
    "tempo": (1, 250),
    "time_signature": (3, 7),
    "valence": (0, 1),
}

# identify anomalies in each feature
anomalies = {}
for feature, (min_val, max_val) in range_checks.items():
    anomalies[feature] = spotify_df_clean.filter((col(feature) < min_val) | (col(feature) > max_val)).count()

# print the anomalies count for each feature
for feature, count in anomalies.items():
    print(f"Number of anomalies in {feature}: {count}")


Number of anomalies in acousticness: 0
Number of anomalies in danceability: 0
Number of anomalies in energy: 0
Number of anomalies in instrumentalness: 0
Number of anomalies in key: 0
Number of anomalies in liveness: 0
Number of anomalies in mode: 0
Number of anomalies in speechiness: 0
Number of anomalies in tempo: 1198
Number of anomalies in time_signature: 13816
Number of anomalies in valence: 0


From the results:

* No Anomalies:
    * Most features (acousticness, danceability, energy, instrumentalness, etc.) have no anomalies. These are clean and can be used as-is.
* tempo Anomalies:
    * 1,198 anomalies fall outside the range [1, 250]. These might include invalid or outlier values (e.g., extremely high or low BPM).
* time_signature Anomalies:
    * 13,816 anomalies fall outside the range [3, 7]. These could represent tracks with unusual time signatures, potentially errors or special cases.

In [24]:
# Show rows with tempo anomalies
tempo_anomalies = spotify_df_clean.filter((col("tempo") < 1) | (col("tempo") > 250))

# Descriptive statistics for tempo anomalies
tempo_anomalies.describe("tempo").show()


+-------+-----+
|summary|tempo|
+-------+-----+
|  count| 1198|
|   mean|  0.0|
| stddev|  0.0|
|    min|  0.0|
|    max|  0.0|
+-------+-----+



In [25]:
# Show rows with time_signature anomalies
time_signature_anomalies = spotify_df_clean.filter((col("time_signature") < 3) | (col("time_signature") > 7))

# Descriptive statistics for time_signature anomalies
time_signature_anomalies.describe("time_signature").show()


+-------+-------------------+
|summary|     time_signature|
+-------+-------------------+
|  count|              13816|
|   mean| 0.9111899247249565|
| stddev|0.28447970650303633|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



**Distinct values**

Let's check the distinct values in the most important columns: `artist_name`, `track_name`, `genre`.

In [22]:
# Count distinct values in key columns
key_columns = ['artist_name', 'track_name', 'genre']
distinct_counts = {col_name: spotify_df.select(col_name).distinct().count() for col_name in key_columns}

for col_name, count in distinct_counts.items():
    print(f"Distinct values in {col_name}: {count}")

Distinct values in artist_name: 64159
Distinct values in track_name: 882285
Distinct values in genre: 392


Since `duration_ms` has no missing values, we will validate for unusual entries like negatives or extreme outliers in the dataset. For example, we will filter out the songs that have negativ or extremely long durations (> 10 minutes). Songs with negativ durations may mean corrupted data. Extremely long songs might need special handling (e.g. audiobooks or podcasts).

In [ ]:
# Validate for negative or extremely high durations (e.g., > 10 minutes in milliseconds)
negative_durations = spotify_df.filter(col("duration_ms") < 0).count()
long_durations = spotify_df.filter(col("duration_ms") > 10 * 60 * 1000).count()

print(f"Number of negativ durations: {negative_durations}")
print(f"Number of extremely long durations (>10 minutes): {long_durations}")

**Quick summary:**

Our group's recommendation system will use Content-Based Filtering method instead of Collaborative filtering. Content-Based Filtering method analyzes the audio characteristics of songs you’ve previously enjoyed, then the model will make personalized suggestions. We do not use Collaborative filtering or user-based filtering method becuase we cannot collect the information related to Spotify's users such as `user_id`.
   
The target variable you are modeling and potentially interesting features of the dataset you will be focusing on during the project. 

**Target variables:**

Based on this focus, we defined that the target variable is `track_id`, which means that we will recommend **tracks** based on their attributes.

**Features:**

We catogorized the features into corresponding groups as follows:

1. Key musical features:

* danceability: Indicates how suitable a track is for dancing.
* energy: Reflects the intensity and activity of the track.
* valence: Captures the musical positivity (happy/sad).
* tempo: The speed or pace of the track in beats per minute.

2. Categorical and context-based features:

* genre: Key for grouping similar tracks into clusters.
* year: Tracks trends and preferences over time.
* popularity: Acts as a secondary metric to boost recommendations for tracks that align with user preferences but are also well-received.

These features are not very important or relevant or already duplicated with other features above. So we might or might not use them for modelling depending on the performance and the needs for feature engineering. 

* acousticness: Degree of acoustic sound in the track.
* instrumentalness: Likelihood of the track being instrumental.
* liveness: Measures the presence of an audience in the track.
* loudness: Reflects the dynamic range and intensity.
* key and mode: Capture musical scale and tonality.
* time_signature: Represents the rhythm structure of the track.


# Data Filtering

# Exploratory Data Analysis (EDA) - IN PROGRESS

In this section, we will explore the dataset to find any patterns and correlations in the dataset. We will use some visualizations to show the trends and also validate the preprocessing in ealier steps.

Here are some tasks that we will perform:

* Check the distribution of numerical features.
* Correlation analysis.
* Analyze the popularity trends over year and by genre.
* Check the distribution by genre.
* Plot the relationship between enegery and danceability.
* Plot the relationship between valence and tempo.
* etc.

## Distribution of numerical features

In this section, we will check the distribution of important numerical features (e.g., popularity, danceability, tempo, valence) to understand data spread and skewness.

More info about each audio feature can be found in [Spotify's API documentation](https://developer.spotify.com/documentation/web-api/reference/get-audio-features).

Reasons to choose these numerical features:

* Popularity: This is a key metric for user preference and relevance. Tracks with higher popularity are more likely to be recommended.
* Danceability: This feature reflects how suitable a track is for dancing—an important factor in user satisfaction for genres like pop or electronic.
* Energy: This feature shows the intensity and activity level of a song, which can help classify songs by mood or genre (e.g., calm vs. energetic).
* Tempo: This is the speed of a song. THis is a key factor for many users when selecting playlists (e.g., playlists for workout, study, or relaxation).
* Valence: This indicates the emotional tone of the song (happy vs. sad), important for personalized recommendations based on mood. 
* Duration (`duration_ms`): This feature is relevant for filtering or grouping songs by length (e.g., short tracks for a quick playlist or long tracks for background music).

In [ ]:
# Select numerical features to visualize
numerical_features = ["popularity", "danceability", "energy", "tempo", "valence", "duration_ms"]

# Convert PySpark DataFrame to Pandas for plotting
numerical_data = spotify_df.select(numerical_features).sample(fraction=0.01).toPandas()

# Plot histograms for each feature
for feature in numerical_features:
    plt.figure(figsize=(6, 4))
    plt.hist(numerical_data[feature].dropna(), bins=30, alpha=0.7)
    plt.title(f"Distribution of {feature}")
    plt.xlabel(feature)
    plt.ylabel("Frequency")
    plt.show()

## Correlation analysis - IN PROGRSS

In [ ]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler
import pandas as pd

# Assemble features into a vector for correlation computation
vector_col = "features_vector"
assembler = VectorAssembler(inputCols=numerical_features, outputCol=vector_col)
spotify_df_vector = assembler.transform(spotify_df.select(numerical_features))

# Compute Pearson correlation matrix
correlation_matrix = Correlation.corr(spotify_df_vector, vector_col, "pearson").head()[0].toArray()

# Convert to Pandas DataFrame for easier visualization
correlation_df = pd.DataFrame(correlation_matrix, columns=numerical_features, index=numerical_features)
print("Correlation Matrix:")
print(correlation_df)

# Visualize the correlation matrix
import seaborn as sns

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_df, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Feature Correlation Heatmap")
plt.show()
